프로그래밍에서 관용구는 특정 작업을 수행하기 위해 코드를 작성하는 특별한 방법이다 매번 동일한 구조를 반복하고 따르는 것이 일반적이다. 디자인 패턴과는 다르다. 가장 큰 차이점은 디자인 패턴을 언어와 무관한 고차원의 개념으로 코드로 즉시 변환되지 않느다. 반면 관용구는 실제 코딩으로 변환된다. 

관용구는 코드이므로 언어에 따라 다르다. 관용구를 따르는 코드를 관용적이라 부르고 특히 파이썬에서는 pythonic이라고 한다.

관용적인 방식으로 코드를 작성했을 때 일반적으로 더 나은 성능을 낸다. 코드도 더 작고 이해하기 쉽다.

전체 개발팀이 동일한 패턴과 구조에 익숙해지면 실수를 줄이고 문제의 본질에 보다 집중할 수 있다.

### 인덱스와 슬라이스

파이썬은 음수 인덱스가 가능하다.

In [2]:
nums = [1, 2, 3, 4, 5]
nums[-1]

5

slice를 이용하여 특정 구간의 요소를 구할 수 있다.

In [3]:
nums[2:5]

[3, 4, 5]

In [4]:
nums[:3]

[1, 2, 3]

In [5]:
nums[3:]

[4, 5]

In [6]:
nums[::]

[1, 2, 3, 4, 5]

In [7]:
nums[1:-1:2]

[2, 4]

위 모든 예제에서 시퀀스에 간격을 전달할 때 실제로는 슬라이스를 전달하는 것과 같다.
슬라이스는 파이썬 내장 객체로 직접 빌드하여 전달할 수도 있다.

In [8]:
interval = slice(1, -1 , 2)
nums[interval]

[2, 4]

In [9]:
interval = slice(None, 3)
nums[interval]

[1, 2, 3]

### 자체 시퀀스 생성

방금 설명한 기능은 "\__getitem\__"이라는 매직 메서드 덕분에 동작한다. 이것은 myobject[key]와 같은 형태를 사용할 때 호출되는 메서드로 key에 해당하는 대괄호 안의 값을 파라미터로 전달한다. 시퀀스는 '\__getitem\__'과 '\__len\__' 모두 구현하는 객체임로 반복이 가능하다. 리스트, 튜플과 문자열은 표준 라이브러리에 있는 시퀀스 객체의 예이다. 

이 섹션에서는 시퀀스나 이터러블 객체를 만들지 않고 키로 객체의 특정 요소를 가져 오는 방법에 대해 다룬다.

클래스가 표준 라이브러리 객체를 감싸는 래퍼인 경우 기본 객체에 가능한 많은 동작을 위임할 수 있다. 

In [11]:
class items:
    def __init__(self, *values):
        self._values = list(values)
    
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

이 예제는 캡슐화 방식을 사용했다. 다른 방법으로 상속을 사용할 수도 있다.

### 컨텍스트 관리자(context manager)

컨텍스트 관리자가 유용한 이유는 패턴에 잘 대응되기 때문이다. 이 패턴은 사실상 모든 코드에 적용될 수 있으며 사전조건과 사후조건을 가지고 있다. 즉 주요 동작의 전후에 작업을 실행하려고 할 때 유용하다.

with 문은 컨텍스트 관리자로 진입하게 한다. 컨텍스트 관리자는 '\__enter\__'와 '\__exit\__' 두 개의 매직 메서드로 구성된다. with문은 '\__enter\__' 메서드를 호출하고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당된다. 사실 '\__enter\__' 메서드가 특정한 값을 반환할 필요는 없다. 설사 값을 반환한다 하더라도 피요하지 않으면 변수에 할당하지 않아도 된다.

해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 이는 파이썬이 처음 호출한 원래 컨텍스트 관리자 객체의 '\__exit\__' 메서드를 호출함을 의미한다. 

In [16]:
def stop_database():
    run("systemctl stop postgresql.service")
    
def start_database():
    run("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()
        
def db_backup():
    run("pg_dump database")
    
def main():
    with DBHandler():
        db_backup()

이 예제에서는 DBHandler를 사용한 블록 내부에서 컨텍스트 관리자의 결과를 사용하지 않았다. 적어도 이 경우에 '\__enter\__'의 반환 값은 쓸모가 없다. 컨텍스트 관리자를 디자인할 때 블록이 시작된 후에 무엇이 필요한지 고려해야 한다. 일반적으로 필수는 아니지만 '\__enter\__'에서 무언가를 반환하는 것이 좋은 습관이다.

main() 함수에서는 유지보수 작업과 상관 없이 백업을 실행한다. 또한 백업에 오류가 있어도 여전히 '\__exit\__'을 호출한다.

'\__exit\__' 메서도의 서명을 주목할 필요가 있다. 블록에서 발생한 예외를 파라미터로 받는다. 블록에 예외가 없으면 모두 None이다.

'\__exit\__'의 반환 값을 잘 생각해야 한다. 특별한 작업을 할 필요가 없다면 아무것도 반환하지 않아도 된다. 만약 '\__exit\__'가 True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 멈춘다는 것을 뜻한다. 때로는 이렇게 하는 것을 원하는 경우도 있지만 일반적으로 발생된 예외를 삼키는 것은 좋지 않은 습관이다. 절대 오류를 조용히 무시해버리면 안 된다는 점을 기억하자.

### 컨텍스트 관리자 구현

앞의 예제와 같은 방법으로 컨텍스트 관리자를 구현할 수 있다. '\__enter\__'와 '\__exit\__' 매직 메서드만 구현하면 해당 객체는 컨텍스트 관리자 프로토콜을 지원할 수 있다. 이렇게 컨텍스트 관리자를 구현하는 것이 일반적인 방법이지만 유일한 방법은 아니다.

컨텍스트 관리자를 좀 더 간결하게 구현하는 방법뿐만 아니라 표준 라이브러리 특히 contextlib 모듈을 사용하여 보다 쉽게 구현하는 방법을 살펴볼 것이다. 

contextlib 모듈은 컨텍스트 관리자를 구현하거나 더 간결한 코드를 작성하는 데 도움이 되는 많은 모우미 함수와 객체를 제공한다. 

먼저 contextmanager 데코레이터를 살펴보자.

함수에 contextlib.contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환한다. 함수는 제너레이터라는 특수한 함수의 형태여야 하는데 이 함수는 코드의 문장을 '\__enter\__'와 '\__exit\__' 매직 메서드로 분리한다. 

지금은 데코레이터와 제너레이터에 익숙하지 않아도 다음에 살펴볼 예제는 특별한 설명 없이도 이해할 수 있고 관용구를 적용하여 이해할 수 있다. 이전 예제와 동일한 코드르 contextmanager 데코레이터를 사용해 다음과 같이 다시 작성할 수 있다.

In [18]:
import contextlib
def run(text):
    print(text)

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

with db_handler():
    db_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


먼저 제너레이터 함수를 정의하고 @contextlib.contextmanager 데코레이터를 적용했다. 이 함수는 yield 문을 사용했으므로 제너레이터 함수가 된다. 여기서도 제너레이터의 상세 내용은 몰라도 된다. 중요한 것은 데코레이터를 적용하면 yield문 앞의 모든 것은 '\__enter\__' 메서드의 일부처럼 취급된다는 것이다. 여기서 생성된 값은 컨텍스트 관리자의 평가 결과로 사용된다. 이번 경우는 yield문에서 아무것도 반환하지 않았다. 이것은 암묵적으로 None을 반환하는 것과 같다. 

이 지점에서 제너레이터 함수가 중단되고 컨텍스트 관리자로 진입하여 데이터베이스의 백업 코드가 실행된다. 이 작업이 완료되면 다음 작업이 이어서 실행되므로 yield문 다음에 오는 모든 것들을 '\__exit\__' 로직으로 볼 수 있다.

이렇게 컨텍스트 매니저를 작성하면 기존 함수를 리팩토링하기 쉬운 장점이 있다. 일반적으로 어느 특정 객체에도 속하는 않는 컨텍스트 관리자가 필요한 경우 좋은 방법이다. 매직 메서드를 추가하면 업무 도메인에 보다 얽히게 되며, 책임이 커지고, 어쩌면 하지 않아도 될 것들을 지원해야만 한다. 많은 상태를 관리ㅏㄹ 필요가 없고 다른 클래스와 독립되어 있는 컨텍스트 관리자 함수를 만드는 경우는 이렇게 하는 것이 좋은 방법이다.

또 다른 도우미 클래스는 contextlib.ContextDecorator이다. 이 클래스는 컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스이다. 반면에 컨텍스트 관리자 자체의 로직은 앞서 언급한 매직 메서드를 구현하여 제공해야 한다. 

In [20]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
    
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

이전 예제와 다른 점은 with문이 없다는 것이다. 그저 함수를 호출하기만 하면 offline_backup 함수가 컨텍스트 관리자 안에서 자동으로 실행된다. 이것이 원본 함수를 래핑하는 데코레이터가 하는 일이다.

이 접근법의 유일한 단점은 완전히 독립적인 것이라는 것이다. 이것은 좋은 특성이다. 데코레이터는 함수에 대해 아무것도 모르고 그 반대도 마찬가지이다. 이것은 좋은 특징이지만 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없다는 것을 의미한다.

데코레이터의 이점은 로직을 한번만 정의하면 동일한 로직을 필요로 하는 함수에 단지 데코레이터를 적용함으로써 원하는 만큼 재사용할 수 있다.

### 프로퍼티, 속성과 객체 메서드의 다른 타입들

public, private, protected 프로퍼티를 가지는 다른 언어들과는 다르게 파이썬 객체의 모든 프로퍼티와 함수는 public이다. 즉 호출자가 객체의 속성을 호출하지 못하도록 할 방법이 없다.

엄격한 강제사항은 없지만 몇 가지 규칙이 있다. 밑줄로 시작하는 속성은 해당 객체에 대해 pricate을 의미하여, 외부에서 호출하지 않기를 기대하는 것이다. 그러나 금지하는 것은 아니다.

파이썬 밑줄의 관습을 알아보자.

### 파이썬에서의 밑줄

In [22]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60

conn = Connector("postgresql://localhost")

'postgresql://localhost'

In [23]:
conn.source

'postgresql://localhost'

In [24]:
conn._timeout

60

In [25]:
conn.__dict__

{'source': 'postgresql://localhost', '_timeout': 60}

_timeout은 connector 차제에서만 사용되고 호출자는 이 속성에 접근하지 않아야 한다. 

timeout 속성은 내부에서만 사용되고 바깥에서는 호출되지 않아서 동일한 인터페이스를 유지하므로 언제든 필요한 경우에 안전하게 리팩토링할 수 있어야 한다. 

이러한 규칙을 준수하면 객체의 인터페이스를 유지하였으므로 파급 효과에 대해 걱정하지 않아도 되기 때문에 유지보수가 쉽고 보다 견고한 코드를 작성할 수 있다. 동이한 원칙이 메서드에도 적용된다.

아래는 객체의 인터페이스를 명확하게 구분하기 위한 파이썬스러운 방식이다. 그러나 일부 속성과 메서드를 실제로 private으로 만들 수 있다는 오해가 있다. 다시 말하지만 이것은 오해이다. 이제 timeoout 속성을 이중 밑줄로 정의했다고 가정해보자. 

In [26]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60
    
    def connect(self):
        print("connecting with {0}s" .format(self.__timeout))
conn = Connector("postgresql://localhost")
conn.connect()

connecting with 60s


In [27]:
conn.__timeout

AttributeError: 'Connector' object has no attribute '__timeout'

일부 개발자는 이 예제와 같은 방법을 사용하여 일부 속성을 숨길 수 있으므로 timeout이 이제 private이며 다른 객체가 수정할 수 없다고 생각하느 경우가 있다. 그러나 위의 에러를 보면 속성이 존재하지 않는다는 의미다. 프라이빗이다 혹은 접근할 수 없다가 아니라 존재하지 않는다고 말한다. 

밑줄 두 개를 사용하면 실제로 파이썬은 다른 이름을 만든다. 이름 맹글링(mangling)이라 한다. "_<class-name\>__<attribute-name\>" 경우

"_Connector__timeout"이라는 속성이 만들어지며 이러한 속성은 다음과 같이 접근하여 수정할 수 있다.

In [28]:
vars(conn)

{'source': 'postgresql://localhost', '_Connector__timeout': 60}

In [29]:
conn._Connector__timeout

60

In [30]:
conn._Connector__timeout = 30

In [31]:
conn.connect()

connecting with 30s


앞에서 언급한 부작용에 의해 속성이 다른 이름으로 존재하는 것에 주목하자. 때문에 앞의 예제에서 속성에러가 발생하는 것이다. 

파이썬에서 이중 밑줄을 사용하는 것은 완전히 다른 경우이다. 여러번 확장되는 클래스의 메서드를 이름 충돌 없이 오버라이드하기 위해 만들어졌다. 

이중 밑줄은 파이썬스러운 코드가 아니다. 속성을 private으로 정의하려는 경우 하나의 밑줄을 사용하고 파이썬스러운 관습을 지키도록 해야 한다.

### 프로퍼티

객체에 값을 저장해야 할 경우 일반적인 속성을 사용할 수 있다. 때로는 객체의 상태나 다른 속성의 값을 기반으로 어떤 계산을 하려고 할 때도 있다. 이런 경우 대부분 프로퍼티를 사용하는 것이 좋은 선택이다.

프로퍼티는 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용한다. 이러게 하는 것 또한 파이썬스러운 코드이다. 자바와 같은 다른 프로그래밍 언어에서는 접근 메서드(getter, setter)를 만들지만 파이썬에서는 프로퍼티를 사용한다. 

사용자가 등록한 정보에 잘못된 정보가 입력되지 않게 보호하려고 한다고 해보자. 예를 들어 다음과 같이 이메일을 입력하는 경우이다.

In [35]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None
        
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

이메일에 프로퍼티를 사용하여 공짜로 몇 가지 이점을 얻을 수 있다. 이 예제에서 첫 번째 @property 메서드는 private 속성인 email 값을 반환한다. 앞에서 언급했듯이 맨 앞에 있는 밑줄은 이 속성이 private으로 사용될 것이므로 외부에서 접근하면 안 된다는 뜻이다.

두 번째 메서트는 앞에서 정의한 프로퍼티에 @email.setter를 추가한다. 이 메서드는 user.email = new_email이 실행될 때 호출되는 코드로 new_email이 파라미터이다. 

In [38]:
u1 = User('llel')
u1.email = "llel@"

ValueError: 유효한 이메일이 아니므로 llel@ 값을 사용할 수 없음

In [39]:
u1.email = 'llel@llel'

In [40]:
u1.email

'llel@llel'

이렇게 하면 get_, set_ 접두어를 사용하여 사용자 메서드를 만드는 것보다 훨씬 더 간단하다. 

단 객체의 모든 속성에 대해 get, set 메서드를 작성할 필요가 없다. 대부분의 경우 일반 속성을 사용하는 것으로 충분하다. 소성 값을 가져오거나 수정할 때 특별한 로직이 필요한 경우에만 프로퍼티를 사용하자.

프로퍼티는 명령-쿼리 분리 원칙을 따르기 위한 좋은 방법이다. 명령-쿼리 분리 원칙은 '객체의 메서드'가 무언가의 상태를 변경하는 커맨드이거나 무언가의 값을 반환하는 쿼리이거나 둘 중 하나만 수행해야지 둘 다 동시에 수행하면 안 된다는 것이다.

@property 데코레이터는 무언가에 응답하기 위한 쿼리이고, @property.setter는 무언가를 하기 위한 커맨드이다. 

한 메서드에서 한 가지 이상의 일을 하지 말라는 것도 알아두자. 유효성 검사를 하고 싶으면 두 개 이상의 문장으로 나누어야 한다. 
한 가지만 수행해야 작업을 처리한 다음 상태를 확인하려면 메서드를 분리해야 한다.

### 이터러블 객체

파이썬에는 기본적으로 반복 가능한 객체가 있다. 리스트 튜플, 세트, 사전은 원하는 구조의 테이터를 보유할 수 있을 뿐 아니라 for문으로 반복적으로 가져올 수 있다. 

그러나 이러한 내장 반복형 객체만 for루프에서 사용 가능한 것은 아니다. 반복을 위해 정의한 로직을 사용해 자체 이터러블을 만들 수도 있다. 엄밀히 말하면 이터러블은 '\__iter__' 매직 메서드를 구현한 객체, 이터레이터는 '\__next__' 매직 메서드를 구현한 객체를 말하는데 지금은 반복과 관련된 객체 정도로만 이해한다.(매직 메서드란? :클래스안에 정의할 수 있는 스페셜 메소드 빌트인 타입(int, str)과 같은 동작을 하게 해준다. 연산자에 대해 데이터 타입에 맞는 메서드로 오버로딩한다. 메서드 이름 앞 뒤로 더블언더스코어를 붙인다.

파이썬의 반복은 이터러블 프로토콜이라는 자체 프로토콜을 사용해 동작한다.

for e in myobj: 형태로 객체를 반복할 수 있는지 확인하기 위해 파이썬은 고수준에서 다음 두 가지를 차례로 검사한다.
    1. 객체가 '__next__'나 '__iter__' 이터레이터 메서드 중 하나를 포함하는지 여부 
    2. 객체가 시퀀스이고 '__len__'과 '__getitem__'를 모두 가졌는지 여부
    
### 이터러블 객체 만들기

객체를 반복하려고 하면 파이썬은 해당 객체의 iter() 함수를 호출한다. 이 함수가 처음으로 하는 것은 해당 객체에 \__iter\__ 메서드가 있는지를 확인하는 것이다. 만약 있으면 \__iter\__ 메서드를 실행한다. 

다음은 일정 기간의 날짜를 하루 간격으로 반복하는 객체의 코드이다.

In [47]:
from datetime import timedelta
from datetime import date
class DateRangeIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._present_day > self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

이 객체는 한 쌍의 날짜를 통해 생성되며 다음과 같이 해당 기간의 날짜를 반복하면서 하루 간격ㅇ로 날짜를 표시한다.

In [48]:
for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05


for 루프는 앞서 만든 객체를 사용해 새로운 반복을 시작한다. 이제 파이썬은 iter() 함수를 호출할 것이고, 이 함수는 \__iter__ 매직 메서드를 호출할 것이다. \__iter__ 메서드는 delf를 반환하고 있으므로 객체 자신이 이터러블임을 나타내고 있다. 따라서 루프의 각 단계에서마다 자신의 next() 함수를 호출한다. next() 함수는 다시 \__next__ 메서드에게 위힘한다. 이 메서드는 요소를 어떻게 생산하고 하나씩 반환할 것인지 결정한다. 더 이상 생상할 것이 없을 경우 파이썬에게 StopIteration 예외를 발생시켜 알려줘야 한다. 즉 for 루프가 작동하는 원리는 StopIteration 예외가 발샐항 때까지 next()를 호출하는 것과 같다.

In [49]:
r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
next(r)

datetime.date(2019, 1, 1)

In [50]:
next(r)

datetime.date(2019, 1, 2)

In [51]:
next(r)

datetime.date(2019, 1, 3)

In [52]:
next(r)

datetime.date(2019, 1, 4)

In [53]:
next(r)

datetime.date(2019, 1, 5)

In [54]:
next(r)

StopIteration: 

이 예제는 잘 동작하지만 작은 문제가 하나 있다. 일단 한 번 실행하면 끝의 날짜에 도달한 상태이므로 이후에 호출하면 계속 StopIteration 예외가 발생한다. 즉, 두 개 이상의 for 루프에서 이 값을 사용하면 철 번째 루프만 작동하고 두 번째 루프는 작동하지 않게 된다. 

In [57]:
r1= DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-05'

In [58]:
max(r1)

ValueError: max() arg is an empty sequence

이 문제가 발생하는 이유는 반복 프로토콜이 작동하는 방식 때문이다. 이터러블 객체는 이터레이터를 생성하고 이것을 사용해 반복을 한다. 위의 예제에서 \__iter__는 self를 반환했지만 호출될 때마다 새로운 이터레이터를 만들 수 있다. 이문제를 수정하는 한 가지 방법은 매번 새로운 인스턴스를 만드는 것이다. 이것도 그리 끔찍한 방법은 아니다. 그러나 \__iter__에서 제너레이터를 사용할 수도 있다. 

In [60]:
class DateRangeContainerIterable:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

이제 다음과 같이 잘 동작한다.

In [63]:
r1= DateRangeContainerIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str, r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [64]:
max(r1)

datetime.date(2019, 1, 4)

달라진 점은 각각의 for루프는 \__iter__를 호출하고, \__iter__는 다시 제너레이터를 생성한다는 것이다. 이러한 형태의 객체를 컨테이너 이터러블이라고 한다.

일반적으로 제너레이터를 사용할 때는 컨테이너 이터러블을 사용하는 것이 좋다.

### 시퀀스 만들기 
객체에 \__iter__ 메서드를 정의하지 않았지만 반복하기를 원하는 경우도 있다. iter() 함수는 객체에 \__iter__가 정의되어 있지 않으면 \__getitem__을 찾고 없으면 TypeError를 발생시킨다.

시퀀스는 \__len__과 \__getitem__을 구현하고 첫 번째 인덱스 0부터 시작하여 포함된 요소를 한 번에 하나씩 차례로 가져올 수 있어야 한다. 즉 \__getitem__을 올바르게 구현하여 이러한 인덱싱이 가능하도록 주의를 기울여야 한다. 그렇지 않으면 반복이 작동하지 않게 된다. 

이전 섹션의 예제는 메모리를 적게 사용한다는 장점이 있다. 즉 한 번에 하나의 날짜만 보관하고 한 번에 하나씩 날짜를 생성하는 법을 알고 있음을 의미한다. 그러나 n번째 요소를 얻고 싶다면 도달할 때까지 n번 반복한다는 단점이 있다. 이 문제는 컴퓨터 과학에서 발생하는 전형적인 메모리와 CPU사이의 트레이트오프이다.

이터러블을 사용하면 메모리를 적게 사용하지만 n 번째 요소를 얻기 위한 시간복잡도는 O(n)이다. 하지만 시퀀스로 구현다면 더 많은 메모리가 사용되지만 특정 요소를 가져오기 위한 인덱싱의 시간복잡도는 O(1)로 상수에 가능하다.

In [71]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
        
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
    
    def __getitem__(self, day_no):
        return self._range[day_no]
        
    def __len__(self):
        return len(self._range())

다음은 이 객체가 어떻게 동작하는지 설명한다.

In [72]:
s1 = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))
for day in s1:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [73]:
s1[-1]

datetime.date(2019, 1, 4)

In [74]:
s1[100]

IndexError: list index out of range

코드를 보면 음수 인덱스도 동작하다는 것을 알 수 있다. 이는 객체가 모든 작업을 래핑된 객체인 리스트에 위임하기 때문인데 이렇게 하는 것이 호환성과 일관성을 유지하는 가장 좋은 방법이다. 

### 컨테이너 객체

컨테이너는 \__contain__ 메서드를 구현한 객체로 \__contain__ 메서드는 일반적으로 Boolean 값을 반환한다. 이 메서드는 파이썬에서 in 키워드가 발견될 때 호출한다.

예를 들면 다음과 같은 코드이다.

In [76]:
element in container

이 코드를 파이썬은 다음과 같이 해석한다.

In [77]:
container.__contain__(element)

NameError: name 'container' is not defined

이 메서드를 잘 사용하면 코드의 가동석이 정말 높아진다. (파이썬스러운 코드다!)

2차원 게임 지도에서 특정 위치에 표시를 해야 한다고 생각해보자. 다음과 같은 함수를 생각할 수 있다.

In [78]:
def mark_coordinate(grid, coord):
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED

여기서 첫 번째 if 문을 상당히 난해해 보인다. 코드의 의도가 무엇인지 이해하기 어렵고 직관적이지 않으며 무엇보다 매번 경계선을 검사하기 위해 if문을 중복해서 호출한다.

지도 자체적으로 grid라 부르는 영역을 판단해주면 어떨까? 그리고 이 일을 더 작은 객체에 위임하면 어떨까? (위임을 통해 응집력도 높아진다.) 이렇게 하면 지도에게 특정 좌표가 포함되어 있는지만 물어보면 된다.

In [79]:
class Bounderies:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height

class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Bounderies(width, height)
        
    def __contains__(self, coord):
        return coord in self.limits

이 코드만으로도 훨씬 효과적인 구현이다. 무엇보다 구성이 간단하고 위임을 통해 문제를 해결한다. 두 객체 모두 최소한의 논리를 사용했고, 메서드는 짭고 응집력이 있다. coord in self.limits는 의도를 직관적으로 표현하여 더 이상의 설명이 필요 없다.

외부에서 사용할 때도 이점이 있다. 다음 고드를 보면 마치 파이썬이 문제를 해결한 것처럼 보인다.

In [80]:
def mark_coordinate(grid, coord):
    if coord in grid:
        grid[coord] = MARKED

### 객체의 동적인 속성

\__getattr__ 매직 메서드를 사용해 객체에서 속성을 얻는 방법을 제어할 수 있다. <myobject>, <myattribute>를 호출하면 파이썬은 객체의 사전에서 <myattribute>를 찾아서 \__getattribute__를 호출한다. 객체에 찾고 있는 속성이 ㅇ벖는 경우 속성(myattribute)의 이름을 파라미터로 전달하여 \__getattr__이라는 추가 메서드가 호출된다. 이 값을 사용하여 반환 값을 제어할 수 있다. 심지어 새로운 속성을 만들 수도 있다.
    

다음은 \__getattr__ 메서드를 설명하는 예제이다.

In [81]:
class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
    
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음.")

다음은 이 클래스 객체애 대한 호출이다.

In [83]:
dyn = DynamicAttributes("value")
dyn.attribute

'value'

In [84]:
dyn.fallback_test

'[fallback resolved] test'

In [85]:
dyn.__dict__["fallback_new"] = "new_value"

In [86]:
dyn.fallback_new

'new_value'

In [88]:
getattr(dyn, "something", "default")

'default'

첫 번째 호출은 간단하다. 객체에 있는 속성을 요청하고 그 결과 값을 반환한다. 두 번째 방법은 객체에 없는 fallback_test라는 메서드를 호출하기 때문에 \__getattr__이 호출되어 값을 반환한다. 이 메서드는 파라미터 값을 포함한 문자열을 반환한다.

세 번째 예제에서 fallback_new라는 새로운 속성이 생성된 것이 흥미롭다. 실제로 이 호출은 dyn.fallback_new = "new value"를 실행한 것과 동일하다. 이 때 \__getattr__의 로직이 적용되지 않은 것에 유의하자. 그 이유는 단순히 메서드가 호출되지 않았기 때문이다. 

마지막은 가장 흥미로운 에이다. 작은 차이가 큰 차이를 만든다. \__getattr__ 메서드를 보면 값을 검색할 수 없는 경우 AttributeError 예외가 발생한다는 것에 유의하자. 이것은 예외 메시지를 포함해 일관성을 유지할 뿐 아니라 내장 getattr() 함수에서도 필요한 부분이다. 이 예외가 발생하면 기본 값을 반환한다.

### 호출형(callable) 객체

함수처럼 동작하는 객체를 정의하면 매우 편리하다. 가장 흔한 사례는 데코레이더를 만드는 것이다. 

매직 메서드 \__call__을 사용하면 객체를 일반 함수처럼 호출할 수 있다. 여기에 전달된 모든 파라미터는 \__call__ 메서드에 그대로 전달된다. 객체를 이렇게 사용하는 주된 이점은 객체는 상태가 있기 때문에 함수 호출 사이에 정보를 저장할 수 있다는 점이다.

파이썬은 object(\*args, \**kwargs)와 같은 구문을 object.\__call__(\*args, \**kwargs)로 변환한다.

이 메서드는 객체를 파라미터가 있는 함수처럼 사용하거나 정보를 기억하는 함ㅅ무처럼 사용할 경우 유용하다.

다음은 입력된 파라미터와 동일한 값으로 몇 번이나 호출되었는지를 반환하는 객체를 만들 때 \__call__ 메서드를 사용하는 예이다.

In [93]:
from collections import defaultdict

class CallCount:
    def __init__(self):
        self._counts = defaultdict(int)
    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]

실제 사용 예를 살펴보자.

In [94]:
cc = CallCount()
cc(1)

1

In [95]:
cc(2)

1

In [96]:
cc(1)

2

In [97]:
cc(1)

3

In [98]:
cc("something")

1

### 파이썬에서 유의할 점

언어의 주요 기능을 이해하는 것 외에도 흔히 발생하는 잠재적인 문제를 피할 수 있는 관용적인 코드를 작성하는 것도 중요하다. 이번 세션에서는 방어코드를 작성하지 않으면 오랜 시간 디버깅하는데 고생할 수 있는 일반적인 이슈들을 살펴본다.

코드 리뷰를 하는 동안 이런 특징을 발견하면 무언가 수정해야 한다는 붕면한 신호이다.

### 변경 가능한 파라미터의 기본 값

변경 가능한 객체를 함수의 기본 인자로 사용하면 안 된다. 만약 변경 가능한 객체를 기본 인자로 사용하면 기대와 다른 결과를 얻게 된다.

다음과 같이 잘못된 함수 정의를 살펴보자.

In [99]:
def wrong_user_display(user_metadata: dict = {'name':'john', "age": 30}):
    name = user_metadata.pop('name')
    age = user_metadata.pop('age')
    return f"{name} ({age})"

여기에는 두 가지 문제가 있다. 반경 가능한 인자를 사용한 것 외에도 함수의 본문에서 가변 객체를 수정하여 부작용이 발생한다. 하지만 가장 큰 문제는 user_metadata의 기본 인자이다.

실제로 이함수는 인자를 사용하지 않고 처음 호출할 때만 동작한다. 다음과 같이 그 다음에 호출할 떄 명시적으로 user_metadata를 전달하지 않으면 KeyError가 발생한다.

In [100]:
wrong_user_display()

'john (30)'

In [101]:
wrong_user_display({"name":"Jane", "age":25})

'Jane (25)'

In [103]:
wrong_user_display()

KeyError: 'name'

이유는 간단하다. 기본 값을 사용해 함수를 호출하면 기본 데이터로 사용될 사전을 한 번만 생성하고 user_metadata는 이것을 가리킨다. 이 값은 프로그램이 실행되는 동안에 계속 메모리에 남아있게 되는데 함수의 본체에서 객체를 수정하고 있다. 이 상태에서 함수의 파라미터에 값을 전달하면 조금 전에 사용한 기본 인자 대신 이 값을 사용한다. 다시 파라미터를 지정하지 않고 기본 값을 사용해 호출하면 실패하게 된다. 왜냐하면 첫 번째 호출 시 키를 지워버렸기 때문이다. 

수정은 간단하다. 기본 초기 값으로 None을 할당하고 함수 본문에서 기본 값을 할당하면 된다. 각 함수는 자체 스코프와 생명주기를 가지므로 None이 나타날 때 마다 user_metadata를 사전에 할당한다.

### 내장(built-in) 타입 확장

리스트, 문자열, 사전과 같은 내장 타입을 확장하는 올바른 방법은 collections 모듈을 사용하는 것이다. 예를 들어 dict를 직접 확장하는 클래스를 만들면 예상하지 못한 결과를 얻을 수 있다. 그 이유는 CPython에서는 클래스의 메서드를 서로 호출하지 않기 때문에  메서드 중에 하나를 오버라이드하면 나머지에는 반영되지 않아서 예기치 않은 결과가 발생한다. 예를 들어 \__getitem__을 오버라이드하고 for루프를 사용해 객체를 반복하려고 하면 해당 로직이 적용되지 않은 것을 알게 된다.

collections.UserDict를 사용하여 문제를 해결할 수 있다. 이렇게 하면 보다 투명한 인터페이스를 제공하고 보다 견고한 코드를 만들 수 있다. 입력 받은 숫자를 접두어가 있는 문자열로 변환하는 리스트를 만든다고 가정해보자. 다음 첫번째 방법은 문제를 해결하는 것처럼 보이지만 오류가 있다.

In [104]:
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"

얼핏 보면 문제가 해결된 것처럼 보인다. 그러나 막상 반복해보면 원하는 것과 다른 결과가 나오게 된다.

In [105]:
bl = BadList((0, 1, 2, 3, 4, 5))
bl[0]

'[짝수] 0'

In [106]:
bl[1]

'[홀수] 1'

In [111]:
"".join(bl)

TypeError: sequence item 0: expected str instance, int found

In [110]:
test_str = ['일','이','삼','사','오','육','칠','팔','구십']
"".join(test_str)

'일이삼사오육칠팔구십'

join은 문자열 리스트를 반복하는 함수이다. BadList의 \__getitem__에서 문자열을 반환했기 때문에 정확히 잘 맞는 문자열의 리스트라고 생각했지만 정확히 반복을 해보면 앞서 정의한 \__getitem__이 호출되지 않는다.

이 문제는 사실 (C에 최적화된) CPython의 세부 구현 사항이며 PyPy와 같은 다른 플랫폼에서는 재현되지 않는다.

그러나 어떤 상황에서도 이식 가능하고 호환 가능한 코드를 작성해야 하므로 리스트가 아니라 UserList에서 확장을 하여 수정하겠다.

In [112]:
from collections import UserList
class BadList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"

In [113]:
bl = BadList((0, 1, 2, 3, 4, 5))
bl[0]

'[짝수] 0'

In [114]:
"".join(bl)

'[짝수] 0[홀수] 1[짝수] 2[홀수] 3[짝수] 4[홀수] 5'